# Assignment 2 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [1]:

def problem1_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    # Define local variables / parameters to satisfy Hull-Dobell
    a = 1103515245
    b = 12345
    period = 2 ** 32
    
    # Generate list of pseudorandom numbers using formula
    x = seed
    pseudo_random_nb = [x % period]
    for i in range(2, size + 1):
        x = (a * x + b) % period
        pseudo_random_nb.append(x)
    
    return pseudo_random_nb

In [2]:

def problem1_uniform(generator=None, period = 2 ** 32, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    # Call the generator method to create a list of pseudorandom numbers
    pseudo_random_nb = generator(size=size, seed=seed)
    
    # Divide each number in the list by M to produce a uniform pseudorandom sequence between 0-1
    uniform_sequence = [x / period for x in pseudo_random_nb]
    
    return uniform_sequence

In [3]:

def problem1_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    import numpy as np
    
    internal_size = 1000
    accept_reject_sample = []
    
    # Set constant M; this is the max of f(x) + tolerance
    m = np.pi / 2 + 0.001
    
    # Generate g(x) - the sampling density (large enough number)
    g_x = uniformGenerator(size=internal_size, seed=seed)
    accept_reject_sample.append(g_x[0])
    
    # Generate f(x) - the target density
    f_x = list(map(lambda x: (np.pi / 2 * abs(np.sin(x * 2 * np.pi))), g_x))
    
    # Draw from U ~ Uniform([0, 1]), using different seed (our proposal distribution)
    u = uniformGenerator(size=internal_size, seed=seed + 3)
    
    # While we have less than the desired number of outputs, generate samples to accept / reject
    for i in range(1, internal_size):
        
        # Calculate r(x), the ratio between them
        r_x = f_x[i] / (m * g_x[i])
        
        # Compare and accept or reject sample
        if u[i] <= r_x:
            accept_reject_sample.append(g_x[i])
            
            if len(accept_reject_sample) == n_iterations:
                break
    
    return accept_reject_sample

---
#### Local Test for Assignment 2, PROBLEM 1
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [4]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem1_LCG(size=10, seed = 1))

period = 2 ** 32

print("Uniform sampler %s" % problem1_uniform(generator=problem1_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem1_uniform(generator=problem1_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem1_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [1, 1103527590, 2524885223, 662824084, 3295386429, 4182499122, 2516284547, 3655513600, 2633739833, 3210001534]
Uniform sampler [2.3283064365386963e-10, 0.25693503906950355, 0.5878706516232342, 0.15432575810700655, 0.767266943352297, 0.9738139626570046, 0.5858681506942958, 0.8511155843734741, 0.6132153405342251, 0.7473867232911289]
Accept-Reject sampler [2.3283064365386963e-10, 0.25693503906950355, 0.5878706516232342, 0.15432575810700655, 0.767266943352297, 0.5858681506942958, 0.8511155843734741, 0.6132153405342251, 0.7473867232911289, 0.06236015981994569, 0.04194940160959959, 0.1948235605377704, 0.1386128985323012, 0.6840353596489877, 0.49171859584748745, 0.7676989699248224, 0.8828409514389932, 0.8232365751173347, 0.8835694054141641, 0.9114757110364735]


In [46]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem1_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.13436424411240122, 0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.4494910647887381, 0.651592972722763, 0.7887233511355132, 0.0938595867742349, 0.02834747652200631, 0.8357651039198697, 0.43276706790505337, 0.762280082457942, 0.0021060533511106927, 0.4453871940548014, 0.7215400323407826, 0.22876222127045265, 0.030589983033553536, 0.0254458609934608, 0.38120423768821243]


---
## Assignment 2, PROBLEM 2
Maximum Points = 8


## Markovian travel

The dataset `Travel Dataset - Datathon 2019` is a simulated dataset designed to mimic real corporate travel systems -- focusing on flights and hotels. The file is at `data/flights.csv` in the same folder as `Exam.ipynb`, i.e. you can use the path `data/flights.csv` from the notebook to access the file.

1. [2p] In the first code-box 
    1. Load the csv from file `data/flights.csv`
    2. Fill in the value of the variables as specified by their names.
2. [2p] In the second code-box your goal is to estimate a Markov chain transition matrix for the travels of these users. For example, if we enumerate the cities according to alphabetical order, the first city `'Aracaju (SE)'` would correspond to $0$. Each row of the file corresponds to one flight, i.e. it has a starting city and an ending city. We model this as a stationary Markov chain, i.e. each user's travel trajectory is a realization of the Markov chain, $X_t$. Here, $X_t$ is the current city the user is at, at step $t$, and $X_{t+1}$ is the city the user travels to at the next time step. This means that to each row in the file there is a corresponding pair $(X_{t},X_{t+1})$. The stationarity assumption gives that for all $t$ there is a transition density $p$ such that $P(X_{t+1} = y | X_t = x) = p(x,y)$ (for all $x,y$). The transition matrix should be `n_cities` x `n_citites` in size.
3. [2p] Use the transition matrix to compute out the stationary distribution.
4. [2p] Given that we start in 'Aracaju (SE)' what is the probability that after 3 steps we will be back in 'Aracaju (SE)'?

In [18]:
import csv
import numpy as np
data = []

with open("data/flights.csv", mode='r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    
    for line in csv_reader:
        data.append(line)
        
data = np.array(data)
        
cities_calc = len(set([sublist[2] for sublist in data]))
userCodes_calc = len(set([sublist[1] for sublist in data]))
observations_calc = len(data)

number_of_cities = cities_calc
number_of_userCodes = userCodes_calc
number_of_observations = observations_calc


In [52]:

# This is a very useful function that you can use for part 2. You have seen this before when parsing the
# pride and prejudice book.

def makeFreqDict(myDataList):
    '''Make a frequency mapping out of a list of data.

    Param myDataList, a list of data.
    Return a dictionary mapping each unique data value to its frequency count.'''

    freqDict = {} # start with an empty dictionary

    for res in myDataList:
        if res in freqDict: # the data value already exists as a key
                freqDict[res] = freqDict[res] + 1 # add 1 to the count using sage integers
        else: # the data value does not exist as a key value
            freqDict[res] = 1 # add a new key-value pair for this new data value, frequency 1

    return freqDict # return the dictionary created

In [12]:

def transition_counts_function(transitions, data):
    
    transition_counts = {}

    for (orig, dest) in transitions:
        mask = (data[:, 2] == orig) & (data[:, 3] == dest)
        travel_count = len(data[mask])
        transition_counts[(orig, dest)] = travel_count
    
    return transition_counts

def transition_matrix_function(unique_cities, transition_counts):

    transition_list = []
    for orig in unique_cities:
        temp_list = []
        for dest in unique_cities:
            if orig != dest:
                temp_list.append(transition_counts[(orig, dest)])
            else:
                temp_list.append(0)

        rel_freq_temp_List = [element / sum(temp_list) for element in temp_list]
        transition_list.append(rel_freq_temp_List)

    transition_matrix = np.array(transition_list)
    
    return transition_matrix

cities = [sublist[2] for sublist in data]
unique_cities = sorted(set(cities)) # The unique cities
n_cities = len(unique_cities) # The number of unique citites

# Count the different transitions
transitions = [(orig, dest) for orig in unique_cities for dest in unique_cities if orig != dest]
# A list containing tuples ex: ('Aracaju (SE)','Rio de Janeiro (RJ)') of all transitions in the text
transition_counts = transition_counts_function(transitions, data) 
# A dictionary that counts the number of each transition 
# ex: ('Aracaju (SE)','Rio de Janeiro (RJ)'):4
indexToCity = {index: city for (index, city) in zip(list(range(n_cities)), unique_cities)} 
# A dictionary that maps the n-1 number to the n:th unique_city,
# ex: 0:'Aracaju (SE)'
cityToIndex = {city: index for (index, city) in zip(list(range(n_cities)), unique_cities)}   
# The inverse function of indexToWord, 
# ex: 'Aracaju (SE)':0

# Part 3, finding the maximum likelihood estimate of the transition matrix

matrix_calc = transition_matrix_function(unique_cities, transition_counts) 
transition_matrix = matrix_calc
# a numpy array of size (n_cities,n_cities)

# The transition matrix should be ordered in such a way that
# p_{'Aracaju (SE)','Rio de Janeiro (RJ)'} = transition_matrix[cityToIndex['Aracaju (SE)'],cityToIndex['Rio de Janeiro (RJ)']]
# and represents the probability of travelling Aracaju (SE)->Rio de Janeiro (RJ)

# Make sure that the transition_matrix does not contain np.nan from division by zero for instance

In [13]:

# This should be a numpy array of length n_cities which sums to 1 and is all positive

eigen_val, eigen_vec = np.linalg.eig(np.transpose(transition_matrix))
first_eigen_vec = eigen_vec[:, 0]
np.array(first_eigen_vec) / np.sum(first_eigen_vec)
stationary = np.array(first_eigen_vec) / np.sum(first_eigen_vec)

stationary_distribution_problem2 = stationary

In [14]:

# Compute the return probability for part 3 of PROBLEM 2

p_0 = np.matrix([1, 0, 0, 0, 0, 0, 0, 0, 0])
p_3 = p_0 * np.matrix(transition_matrix) ** 3
    
p_3_aracuja = p_3[0, 0]

return_probability_problem2 = p_3_aracuja

---
#### Local Test for Assignment 2, PROBLEM 2
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [15]:
# Once you have created all your functions, you can make a small test here to see
# what would be generated from your model.
import numpy as np

start = np.zeros(shape=(n_cities,1))
start[cityToIndex['Aracaju (SE)'],0] = 1

current_pos = start
for i in range(10):
    random_word_index = np.random.choice(range(n_cities),p=current_pos.reshape(-1))
    current_pos = np.zeros_like(start)
    current_pos[random_word_index] = 1
    print(indexToCity[random_word_index],end='->')
    current_pos = (current_pos.T@transition_matrix).T

Aracaju (SE)->Brasilia (DF)->Natal (RN)->Florianopolis (SC)->Campo Grande (MS)->Florianopolis (SC)->Aracaju (SE)->Campo Grande (MS)->Aracaju (SE)->Natal (RN)->

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment3Problem1(x)`, where `x` is a list of $n$ input data points.

The likelihood function $f(x; \lambda)$ is given by

$$
    L_n(\lambda) = \prod_{i=1}^n f(x_i ; \lambda) = \prod_{i=1}^n \frac{1}{24} \lambda^5 x_i^4 e^{(-\lambda x_i)}
$$

Taking the log likelhood we get

$$
    l_n(\lambda) = \sum_{i=1}^n log(f(x_i ; \lambda)) = 
    \sum_{i=1}^n log(1/24) + log(\lambda^5) + log(x^4) - \lambda x \, log(e) =
    \sum_{i=1}^n log(1/24) + 5 \, log(\lambda) + 4 \, log(x) - \lambda x
$$

Let $r = log(1/24) + 5 log(\lambda) + 4 log(x) - \lambda x$ and take the partial derivative with respect to $\lambda$, then set this to $0$ to obtain a maximum point (everything that doesn't involve $\lambda$ is a constant that disappears):

$$
    \frac{\partial}{\partial \lambda} r = \frac{\partial}{\partial \lambda} log(1/24) + 5 \, log(\lambda) + 4 \, log(x) - \lambda x
    = \frac{5}{\lambda} - x
$$


$$
    \frac{5}{\lambda} - x = 0 \Longleftrightarrow \frac{5}{\lambda} = x \Longleftrightarrow 5 = \lambda x
    \Longleftrightarrow \lambda = \frac{5}{x}
$$

The MLE is then given by the sum of $\frac{5}{x}$ across all samples in the list $x$.

$$
    MLE = \sum_{i=1}^{n} \frac{5}{x_i} = \frac{5n}{\sum_{i=1}^n x_i} = 5n \sum_{i=1}^{n} \frac{1}{x_i}
$$

In [23]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''write comment of what this function does'''
    mle = (5 * len(x)) / sum(x)
    return mle

---
## Assignment 2, PROBLEM 4
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example above (in `09.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [33]:


import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    
    mu_param=parameters[0]
    sigma_param=parameters[1]
    
    def f_normal(x, sigma_param=sigma_param, mu_param=mu_param):
        f_normal = (1 / (sigma_param * np.sqrt(2 * np.pi))) * np.exp((-1 / (2 * sigma_param ** 2)) * (x - mu_param) ** 2)
        f_normal_log = np.log(f_normal)
        return f_normal_log
    
    log_likelihood = -sum([f_normal(x) for x in normallySimulatedDataSamples])
    
    return log_likelihood

# you should only change XXX below and not anything else
parameter_bounding_box=((-20, 20), (0.1, 5)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([5, 2.6]) # point in 2D to initialise the minimize algorithm
result_Ass2Prob4 = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box) 
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_Ass2Prob4


      fun: 58.631387282367946
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.13162803e-06,  6.39488466e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 36
      nit: 7
     njev: 12
   status: 0
  success: True
        x: array([9.26861952, 1.70820175])